# Level 2

We have discussed [Level 1](onboarding_level1.ipynb) in great detail. As we have seen, Level 1 is relatively simple.

In Level 2, we see more flexibility and additional choices. You will be exposed to some simple Python code and see how Aviary establishes a problem from beginning to end. Each step is built upon the capabilities of [Level 3](onboarding_level3). Users will be led to Level 3 in a natural way.

Level 2 is defined in the [aviary/interface/methods_for_level1.py file](https://github.com/OpenMDAO/Aviary/blob/main/aviary/interface/methods_for_level1.py) and it has a single method `run_aviary()` with a few arguments. If you examine `interface/level1.py` you see that Level 1 prepares those arguments and then call `run_aviary()`. For `aviary run_mission aircraft_for_bench_GwGm` examples, those arguments are:

- `aircraft_filename`: `aircraft_for_bench_GwGm`
- `phase_info`: `phase_info` (loaded from `aviary/interface/default_phase_info/gasp.py`)
- `optimizer`: `IPOPT` (the default is `None`)
- `analysis_scheme`: `AnalysisScheme.COLLOCATION` (the default)
- `objective_type`: `None` (default)
- `record_filename`: `aviary_history.db` (the default)
- `restart_filename`: `None` (the default)
- `max_iter`: 50 (the default)

All the above arguments are straightforward except `objective_type`. Even though `objective_type` is `None`, it is not treated as `None`. In this scenario, the objective is set based on `problem_type` when using the GASP-based mission method, but not the FLOPS-based mission. There are three options for `problem_type`. Aviary has the following mapping when `objective_type` is not set by user and `mission_method` is `GASP`:

| problem_type | objective |
| ------------ | --------- |
| `SIZING` | Mission.Objectives.FUEL |
| `ALTERNATE` | Mission.Objectives.FUEL |
| `FALLOUT` | Mission.Objectives.RANGE |

In Aviary, `problem_type` is set to `SIZING` when it creates a vehicle (see [create_vehicle](https://github.com/OpenMDAO/Aviary/blob/main/aviary/utils/UI.py)). As you can see, since `problem_type` is `SIZING` by default in our case and we don't manually alter this setting, Aviary set objective to `Mission.Objectives.FUEL`. We will discuss more options of `objective_type` later on.

In our onboarding runs, we want to limit the number of iterations to 1 so that they all run faster. As a result, we will not consider whether the optimization converges. So, we will have

```
`max_iter`: 1
```

Level 2 cannot be run via `aviary` command on the command line. Users must develop level 2 Python code. The good news is that the Python code is pretty small. You can follow the following steps in order (we do not include function arguments for simplicity):

- `prob = AviaryProblem()`
- `prob.load_inputs()`
- `prob.check_and_preprocess_inputs()`
- `prob.add_pre_mission_systems()`
- `prob.add_phases()`
- `prob.add_post_mission_systems()`
- `prob.link_phases()`
- `prob.add_driver()`
- `prob.add_design_variables()`
- `prob.add_objective()`
- `prob.setup()`
- `prob.set_initial_guesses()`
- `prob.run_aviary_problem()`

In the rest of this page, we will show a few examples to demonstrate how level 2 runs these steps. We start from rebuilding `aircraft_for_bench_GwGm` model in great details.

## Build level 2 for the same `aircraft_for_bench_GwGm` Model

We create a level 2 Python script to reproduce the `aircraft_for_bench_GwGm` model run that was used as an example in the level 1 document (this time we won’t use the level 1 functionality). The methods listed above are defined in level 3 (namely, [interface/methods_for_level2.py](https://github.com/OpenMDAO/Aviary/blob/main/aviary/interface/methods_for_level2.py)). You can run the code as follows:

In [ ]:
from aviary.api import Aircraft, Mission
import aviary.api as av
from copy import deepcopy


# inputs that run_aviary() requires
aircraft_filename = "models/test_aircraft/aircraft_for_bench_GwGm.csv"
optimizer = "IPOPT"
analysis_scheme = av.AnalysisScheme.COLLOCATION
objective_type = None
record_filename = 'aviary_history.db'
restart_filename = None
max_iter = 0
phase_info = deepcopy(av.default_2DOF_phase_info)

# Build problem
prob = av.AviaryProblem(analysis_scheme)

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs(aircraft_filename, phase_info)

# Preprocess inputs
prob.check_and_preprocess_inputs()

# adds a pre-mission group (propulsion, geometry, static aerodynamics, and mass)
prob.add_pre_mission_systems()

# adds a sequence of core mission phases.
prob.add_phases()

# adds a landing phase
prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

# adds an optimizer to the driver
prob.add_driver(optimizer, max_iter=max_iter)

# adds relevant design variables
prob.add_design_variables()

# Load optimization problem formulation
# Detail which variables the optimizer can control
prob.add_objective(objective_type=objective_type)

# setup the problem
prob.setup()

# set initial guesses of states and controls variables
prob.set_initial_guesses()

# run the problem we just set up
prob.run_aviary_problem(record_filename, restart_filename=restart_filename)

In this code, you do the same import as `methods_for_level1.py` does and set the values of all the arguments in `run_aviary()`. Now we will go through each line in detail to explain each step:

## Dissection of level 2 for the same `aircraft_for_bench_GwGm` model

All the methods of `prob` object (including its creation) are defined in level 3 (`methods_for_level2.py`). We now look at each of them.

We add other inputs that `run_aviary()` requires:

In [ ]:
aircraft_filename = "models/test_aircraft/aircraft_for_bench_GwGm.csv"
optimizer = "IPOPT"
analysis_scheme = av.AnalysisScheme.COLLOCATION
objective_type = None
record_filename = 'aviary_history.db'
restart_filename = None
max_iter = 1

prob = av.AviaryProblem(analysis_scheme)

Several objects are initialized in this step:

```
self.model = om.Group()
self.pre_mission = PreMissionGroup()
self.post_mission = PostMissionGroup()
self.aviary_inputs = None
self.phase_info = phase_info
self.traj = None
self.analysis_scheme = analysis_scheme
self.pre_mission_info = self.phase_info.pop('pre_mission')
self.post_mission_info = self.phase_info.pop('post_mission')
```

`phase_info` is a user defined dictionary (in a Python file) that controls the profile of the mission to be simulated (e.g. climb, cruise, descent segments etc).

For `analysis_scheme`, the two options are: `AnalysisScheme.COLLOCATION` (default) and `AnalysisScheme.SHOOTING` which are defined and described in [variables_info/enums.py`](https://github.com/OpenMDAO/Aviary/blob/main/aviary/variable_info/enums.py):
- COLLOCATION uses the collocation method to optimize all points simultaneously.
- SHOOTING is a forward in time integration method that simulates the trajectory.

In this onboarding document, only the `COLLOCATION` scheme will be discussed. The line

In [ ]:
phase_info = deepcopy(av.default_2DOF_phase_info)
prob.load_inputs(aircraft_filename, phase_info)

is a function that has a few tasks:

- Read aircraft deck file `aircraft_filename`
- Read phase info file `phase_info`
- Build core subsystems

We have seen `aircraft_filename` file (a `.csv` file) in our level 1 examples. In [level 1](onboarding_level1), we simply called it input file. An aircraft model can also be directly defined in Python, by setting up an `AviaryValues` object with the desired inputs and options normally found in an input file. That object can be provided in the place of `aircraft_filename`.


Engines are built by using `aircraft:engine:data_file` in the .csv file. For example in `aircraft_for_bench_GwGm.csv` file, we see:

```
aircraft:engine:data_file,models/engines/turbofan_28k.deck,unitless
```

So, `aircraft:engine:data_file` has value `models/engines/turbofan_28k.deck,unitless`. The top rows of engine deck file are:

| **Mach_Number (unitless)** | **Altitude (ft)** | **Throttle (unitless)** | **Gross_Thrust (lbf)** | **Ram_Drag (lbf)** | **Fuel_Flow (lb/h)** | **NOx_Rate (lb/h)** |
| ---------------------- | -------------- | -------------------- | ------------------- | --------------- | --- | --- |
| 0.0, | 0.0, | 50.0, | 28928.1, | 0.0, | 8662.3, | 61.9894 |
| 0.0, | 0.0, | 48.0, | 26999.7, | 0.0, | 7932.6, | 49.2185 |
| 0.0, | 0.0, | 46.0, | 25071.1, | 0.0, | 7258.1, | 33.3976 |
| 0.0, | 0.0, | 42.0, | 21214.0, | 0.0, | 5979.1, | 19.8547 |
| 0.0, | 0.0, | 38.0, | 17356.9, | 0.0, | 4795.2, | 17.5877 |

Users can provide an `EngineModel` instance of their own to use in Aviary's propulsion systems by adding it to `engine_models`.

Other subsystems, including mass, geometry, and aerodynamics, are set up according to which legacy code options the user has specified in their input file, using `settings:equations_of_motion` and `settings:mass_method`. Aerodynamics is set up to match the selected equations of motion, while geometry will use either GASP, FLOPS, or both methods as required to calculate all values needed by other subsystems.

Next we check the user-provided inputs:

In [ ]:
prob.check_and_preprocess_inputs()

This method checks the user-supplied input values for any potential problems. These problems include variable names that are not recognized in Aviary, conflicting options or values, or units mismatching.

Next, we add pre-mission systems:

In [ ]:
prob.add_pre_mission_systems()

This call adds a pre-mission group (also called static analysis group) which includes pre-mission propulsion, geometry, pre-mission aerodynamics, and mass subsystems. 

For `height_energy` missions, aviary currently models FLOPS' "simplified" takeoff as defined in [mission/flops_based/phases/simplified_takeoff.py](https://github.com/OpenMDAO/Aviary/blob/main/aviary/mission/flops_based/phases/simplified_takeoff.py).

Next is the line

In [ ]:
prob.add_phases()

which adds a sequence of core mission phases. In addition, if `mission_method` is `2dof` and `ascent` is a phase, it adds an equality constraint to the problem to ensure that the TAS at the end of the groundroll phase is equal to the rotation velocity at the start of the rotation phase (`_add_groundroll_eq_constraint(phase)`). If `mission_method` is `height_energy`, it sets up trajectory parameters by calling `setup_trajectory_params()`. If `mission_method` is `solved`, it has a block of code to make sure that the trajectory is smooth by applying boundary constraints between phases (e.g. fuselage pitch angle or true airspeed).

It follows by adding post-mission subsystems:

In [ ]:
prob.add_post_mission_systems()

Similar to pre-mission, it adds a landing phase if `include_landing` key of `post_mission` has value of `True`. If user chooses to define a `post_mission`, it will override the default. For `2dof` missions, landing is defined in [mission/gasp_based/phases/landing_group.py](https://github.com/OpenMDAO/Aviary/blob/main/aviary/mission/gasp_based/phases/landing_group.py). For `simple` mission, landing means a [simplified landing](https://github.com/OpenMDAO/Aviary/blob/main/aviary/mission/flops_based/phases/simplified_landing.py). Note that the `solved` method currently doesn't have any post mission systems.

The next line is

In [ ]:
prob.link_phases()

This is important for allowing each phase of flight to pass to the next without discontinuities in the parameters. Consider Dymos' [Aircraft Balanced Field Length Calculation](https://openmdao.github.io/dymos/examples/balanced_field/balanced_field.html) example. In that example, we see separate nonlinear boundary constraints, nonlinear path constraints, and phase continuity constraints between phases. We don't want to go deeper in this function call, but just point out that each individual link can be set via dymos function `link_phases`. See [dymos API](https://openmdao.github.io/dymos/api/trajectory_api.html) for more details.

The code blocks in this function (namely, `link_phases()`) are for `2DOF`, `simple`, and `solved` missions. The links are set up based on physical principals (e.g. you can’t have instantaneous changes in mass, velocity, position etc.). Special care is required if the user selects a different or unusual set of phases. 

Now, our aircraft and the mission are fully defined. We are ready to define an optimization problem. This is achieved by adding an optimization driver, adding design variables, and an objective. 

For `add_driver` function, we accept `use_coloring=None`. Coloring is a technique that OpenMDAO uses to compute partial derivatives efficiently. This will become important later.

In [ ]:
prob.add_driver(optimizer, max_iter=max_iter)

Drivers available for use in Aviary are `SLSQP`, `SNOPT`, and `IPOPT`. The table below summarizes the basic setting along with sample values (the settings are options required by each optimizer):

| **Optimizers** | **Drivers** | **Settings** |
| ---------- | ------- | -------- |
| `SNOPT` | om.pyOptSparseDriver() | `Major iterations limit`: 50<BR/>`Major optimality tolerance`: 1e-4<BR/>`Major feasibility tolerance`: 1e-6<BR/>`iSumm`: 6 |
| `IPOPT` | om.pyOptSparseDriver() | `tol`: 1e-9<BR/>`mu_init`: 1e-5<BR/>`max_iter`: 50<BR/>`print_level`: 5 |
| `SLSQP` | om.ScipyOptimizeDriver() | `tol`: 1.0E-9<BR/>`maxiter`: 50<BR/>`disp`: True |

Note that `SLSQP` is freely available, but its performance is not as good as `SNOPT` and `IPOPT` sometimes. `SNOPT` is a commercial optimizer, but it is free for academic use. `IPOPT` is an open-source optimizer and it is free for all users.

Design variables (and constraints) are set in the line `prob.add_design_variables()`:

In [ ]:
prob.add_design_variables()

For default `FLOPS` mission model, it is relatively simple:

| **Design Variables** | **Lower Bound** | **Upper Bound** | **Reference Value** | **Units** |
| ----------- | ----------- | ----------- | --------------- | ----- |
| Mission.Design.GROSS_MASS | 100.e3 | 200.e3 | 135.e3 | lbm |

For default `GASP` mission model, the design variables and constraints depend on the type of problems (`SIZING`, `ALTERNATE`, or `FALLOUT`, see `ProblemType` class in `aviary/variable_info/enums.py` for details). First, there are four common design variables and two common constraints. There are two more design variables and two constraints for sizing problem.

| **Problem Type** | **Design Variables** | **Lower Bound** | **Upper Bound** | **Reference** Value | Units |
| ----------- | ----------- | ----------- | ----------- | --------------- | ----- |
| Any | Mission.Takeoff.ASCENT_T_INITIAL | 0 | 100  | 30.0 | s |
| Any | Mission.Takeoff.ASCENT_DURATION  | 1 | 1000 | 10.0 | s |
| Any | tau_gear  | 0.01 | 1.0 | 1 | s |
| Any | tau_flaps | 0.01 | 1.0 | 1 | s |
| SIZING | Mission.Design.GROSS_MASS | 10. | 400.e3 | 175_000 | lbm |
| SIZING | Mission.Summary.GROSS_MASS | 10. | 400.e3 | 175_000 | lbm |
| ALTERNATE | Mission.Summary.GROSS_MASS | 0 | infinite | 175_000 | lbm |
| **Problem Type** | **Constraint** | **Relation** | **Value** | **Reference Value** | **Units** |
| Any | h_fit.h_init_gear | = | 50.0  | 50.0 | ft |
| Any | h_fit.h_init_flaps | = | 400.0 | 400.0 | ft |
| SIZING | Mission.Constraints.RANGE_RESIDUAL | = | 0 | 10 | unitless |
| ALTERNATE | Mission.Constraints.RANGE_RESIDUAL | = | 0 | 10 | lbm |

In the above table, there are two hard-coded design variables: `tau_gear` and `tau_flaps`. They represent fractions of ascent time to start gear retraction and flaps retraction. There are two hard-coded constraints: `h_fit.h_init_gear` and `h_fit.h_init_flaps`. They are the altitudes of initial gear retraction and initial flaps retraction. The underscore in number "175_000" is for readability only. 

There are other constraints using OpenMDAO's `EQConstraintComp` component. We will not go into the details as this part is complicated and needs special attention. Note that each subsystem (for example engine model) might have their own design variables (think, for example, sizing the engine). Aviary goes through all subsystems and adds appropriate design variables.

You can override all the functions in level 3. So, you can set up your constraints in level 3 if the above ones do not meet your requirements.

The optimization objective is added to the problem by this line:

In [ ]:
prob.add_objective(objective_type=objective_type)

The selection of objective is a little complicated. 

Earlier in this page, we have discussed the objective when `objective_type=None` and `mission_method` is `2DOF`. Let us discuss the other situations.

There are several objective types that users can choose: `mass`, `hybrid_objective`, `fuel_burned`, and `fuel`. 

| objective_type | objective |
| -------------- | --------- |
| mass | `Dynamic.Mission.MASS` |
| hybrid_objective | `-final_mass / {takeoff_mass} + final_time / 5.` |
| fuel_burned | `initial_mass - mass_final` (for `FLOPS` mission only)|
| fuel | `Mission.Objectives.FUEL` |

As listed in the above, if `objective_type="mass"`, the objective is the final value of `Dynamic.Mission.MASS` at the end of the mission.
If `objective_type="fuel"`, the objective is the `Mission.Objectives.FUEL`.
There is a special objective type: `hybrid_objective`. When `objective_type="hybrid_objective"`, the objective is a mix of minimizing fuel burn and minimizing the mission duration:

```
      obj = -final_mass / {takeoff_mass} + final_time / 5.
```
This is because if we just minimized fuel burn then the optimizer would probably fly the plane slowly to save fuel, but we actually care about some mix of minimizing fuel burn while providing a reasonable travel time for the passengers. This leads to the `hybrid_objective` which seeks to minimize a combination of those two objectives. `final_time` is the duration of the full mission and is usually in the range of hours. So, the denominator `5.` means `5 hours`. That's just a value to scale the final_time variable. Since it's a composite objective we didn't want to have OpenMDAO do the scaling because the two variables in the objective are of a different order of magnitude. We will show an example of this objective type in level 2 discussion.

If `objective_type=None` for a `2DOF` mission, Aviary will choose the objective based on `mission_method` and `problem_type`. We have discussed this case earlier in this page.

If `objective_type=None` for a `FLOPS` mission method, Aviary adds a `fuel_burned` objective. In fact, this objective is available for `FLOPS` mission only. That number you get is for the actual fuel burned in lbm. You may get something like `fuel_burned` with the scaled value `[3.91228276]`, reference value `1e+04`. It translates to the dimensional physical quantity of `39,122.8276` lbm. To see this in action, check the resulting `reports/opt_report.html` file to see the optimal results.

**Note:**  Aviary variable `Mission.Objectives.FUEL` when using the GASP-based mission is actually a hybrid objective defined as

```
      reg_objective = overall_fuel/10000 + ascent_duration/30.
```
where `overall_fuel` has the unit of `lbm` and `ascent_duration` has the unit of seconds. In our case, `settings:equations_of_motion = 2DOF`, the final value of objective is `[5.5910123]`, with `ref: 1.0` and `units: blank`. The units should be interpreted as `unitless`.

Here, `ref` is the reference value. For different objectives, the range may vary significantly different. We want to normalize the value. Ideally, users should choose `ref` such that the objective is in the range of `(0,1)`. This is required by optimizer.

**Note:**  Unfortunately, not all `objective_type` and `mission_method` combinations work.

Next is a line to call

In [ ]:
prob.setup()

This is a lightly wrapped OpenMDAO `setup()` method for the problem. It allows us to do `pre-` and `post-setup` changes, like adding calls to `set_input_defaults` and do some simple `set_vals` if needed. 

If we look at the signature of `setup()` in OpenMDAO's [Problem](https://openmdao.org/newdocs/versions/latest/_srcdocs/packages/core/problem.html) class, we find that the available kwargs are: `check`, `logger`, `mode`, `force_alloc_complex`, `distributed_vector_class`, `local_vector_class`, and `derivatives`. The ones that Aviary uses are `check` and `force_alloc_complex`. Argument `check` is a flag to determine default checks are performed. [Default checks](https://openmdao.org/newdocs/versions/latest/theory_manual/setup_stack.html) are: 'auto_ivc_warnings', comp_has_no_outputs', 'dup_inputs', 'missing_recorders', 'out_of_order', 'solvers', 'system', 'unserializable_options'.

If [force_alloc_complex](https://openmdao.org/newdocs/versions/latest/advanced_user_guide/complex_step.html) is true, sufficient memory will be allocated to allow nonlinear vectors to store complex values while operating under complex step. For our example, we don't use any of them.

For optimization problems, initial guesses are important.

In [ ]:
prob.set_initial_guesses()

For `simple` and `2DOF` missions, this method performs several calls to `set_val` on the trajectory for states and controls to seed the problem with reasonable initial guesses using `initial_guesses` within corresponding phases (e.g. `default_flops_phases.py` and `default_gasp_phases.py`). For `solved` missions, it performs similar tasks but for hard-coded state parameters. This is reasonable because a `solved` mission is actually a level 3 Aviary approach. We will cover it in [level 3 onboarding doc](onboarding_level3.ipynb) in the next page. Note that initial guesses for all phases are especially important for collocation methods.

The last line is to run the problem we just set up:

In [ ]:
prob.run_aviary_problem()

This is a simple wrapper of Dymos' [run_problem()](https://openmdao.github.io/dymos/api/run_problem_function.html) function. It allows the users to provide `record_filename`, `restart_filename`, `suppress_solver_print`, and `run_driver`. In our case, `record_filename` is changed to `aviary_history.db` and `restart_filename` is set to `None`. The rest of the arguments take default values. If a restart file name is provided, aviary (or dymos) will load the states, controls, and parameters as given in the provided case as the initial guess for the next run. We have discussed the `.db` file in [level 1 onboarding doc](onboarding_level1.ipynb) and will discuss how to use it to generate useful output in [level 3 onboarding doc](onboarding_level3.ipynb).

Finally, we can add a few print statements for the variables that we are interested:


In [ ]:
print("Mission.Objectives.FUEL",
      prob.get_val(Mission.Objectives.FUEL, units='unitless'))
print("Mission.Design.FUEL_MASS",
      prob.get_val(Mission.Design.FUEL_MASS, units='lbm'))
print("Mission.Design.FUEL_MASS_REQUIRED",
      prob.get_val(Mission.Design.FUEL_MASS_REQUIRED, units='lbm'))
print("Mission.Summary.TOTAL_FUEL_MASS",
      prob.get_val(Mission.Summary.TOTAL_FUEL_MASS, units='lbm'))
print("Mission.Summary.GROSS_MASS (takeoff_mass)",
      prob.get_val(Mission.Summary.GROSS_MASS, units='lbm'))
print("Mission.Landing.TOUCHDOWN_MASS (final_mass)",
      prob.get_val(Mission.Landing.TOUCHDOWN_MASS, units='lbm'))
print()

print("Groundroll Final Mass (lbm)",
      prob.get_val('traj.phases.groundroll.states:mass', units='lbm')[-1])
print("Rotation Final Mass (lbm)",
      prob.get_val('traj.rotation.states:mass', units='lbm')[-1])
print("Ascent Final Mass (lbm)",
      prob.get_val('traj.ascent.states:mass', units='lbm')[-1])
print("Accel Final Mass (lbm)",
      prob.get_val('traj.accel.states:mass', units='lbm')[-1])
print("Climb1 Final Mass (lbm)",
      prob.get_val('traj.climb1.states:mass', units='lbm')[-1])
print("Climb2 Final Mass (lbm)",
      prob.get_val('traj.climb2.states:mass', units='lbm')[-1])
print("Cruise Final Mass (lbm)",
      prob.get_val('traj.phases.cruise.rhs.calc_weight.mass', units='lbm')[-1])
print("Desc1 Final Mass (lbm)",
      prob.get_val('traj.desc1.states:mass', units='lbm')[-1])
print("Desc2 Final Mass (lbm)",
      prob.get_val('traj.desc2.states:mass', units='lbm')[-1])
print('done')

We will cover user customized outputs in [level 3](onboarding_level3.ipynb).

## Level 2: Another example

We now use a similar aircraft, a large single aisle commercial transport aircraft, but with a different mass estimation and mission method. Let us run Aviary using this input deck in level 1 first.

In [ ]:
!aviary run_mission models/test_aircraft/aircraft_for_bench_FwFm.csv --max_iter 0 --optimizer IPOPT

Once again, to convert it to a level 2 model, we need to set all the arguments in level 1 manually.

By running a model in level 2 directly, we have the flexibility to modify the input parameters (e.g. `phase_info`). Let us continue to make modifications and obtain a different run script shown below:

In [ ]:
phase_info = {
    'pre_mission': {
        'include_takeoff': False,
        'optimize_mass': False,
    },
    'cruise': {
        'subsystem_options': {
            'core_aerodynamics': {'method': 'computed'}
        },
        'user_options': {
            'optimize_mach': False,
            'optimize_altitude': False,
            'polynomial_control_order': 1,
            'num_segments': 2,
            'order': 3,
            'solve_for_distance': False,
            'initial_mach': (0.72, 'unitless'),
            'final_mach': (0.72, 'unitless'),
            'mach_bounds': ((0.7, 0.74), 'unitless'),
            'initial_altitude': (35000.0, 'ft'),
            'final_altitude': (35000.0, 'ft'),
            'altitude_bounds': ((23000.0, 38000.0), 'ft'),
            'throttle_enforcement': 'boundary_constraint',
            'fix_initial': True,
            'constrain_final': False,
            'fix_duration': False,
            'initial_bounds': ((0.0, 0.0), 'min'),
            'duration_bounds': ((10., 30.), 'min'),
        },
        'initial_guesses': {'times': ([0, 30], 'min')},
    },
    'post_mission': {
        'include_landing': False,
    },
}

# inputs that run_aviary() requires
aircraft_filename = "models/test_aircraft/aircraft_for_bench_FwFm.csv"
mission_method = "height_energy_energy"
mass_method = "FLOPS"
optimizer = "SLSQP"
analysis_scheme = av.AnalysisScheme.COLLOCATION
objective_type = None
record_filename = 'history.db'
restart_filename = None

# Build problem
prob = av.AviaryProblem(analysis_scheme)

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs(aircraft_filename, phase_info)

# Preprocess inputs
prob.check_and_preprocess_inputs()

prob.add_pre_mission_systems()

prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

prob.add_driver(optimizer, max_iter=0)

prob.add_design_variables()

# Load optimization problem formulation
# Detail which variables the optimizer can control
prob.add_objective(objective_type=objective_type)

prob.setup()

prob.set_initial_guesses()

prob.run_aviary_problem(record_filename)

print("done")


As you see, there is a single phase `cruise`, no takeoff, no landing. Note that we must set `include_takeoff` to `False` because Aviary internally tries to connect takeoff to climb phase which we don't provide. There should be a check to see if both takeoff and climb phase exist first. Aviary still has many things to be improved.

We will see more details for what users can do in [level 3](onboarding_level3.ipynb).

Level 2 is where you can integrate user-defined [external subsystems](../user_guide/subsystems.md), which is one of the main features of the Aviary tool. [Examples](../user_guide/using_external_subsystems.md) of external subsystems are: acoustics, battery modeling, etc.
Assume that you already have an external subsystem that you want to incorporate it into your model. We show how to add external subsystems via `external_subsystems` key in `phase_info`.

We will cover external subsystems in details in [Models with External Subsystems](onboarding_ext_subsystem.ipynb) page.


## Summary

As you see, level 2 is more flexible than level 1. In level 2, you can:
- add/remove pre-defined mission phases (via `phase_info`, see example above);
- scale design variables (via reference value in `phase_info`)
- import additional files (e.g. `aero_data_file`);
- set pre-defined objective (e.g. `hybrid_objective`);
- add external subsystems (via `phase_info`);
- set `use_coloring` (see example above).

Most Aviary users should be well-served by Level 2; we have purposefully constructed it to be capable of most all use cases, even those on the forefront of research in aircraft design.

That being said, there are some cases where Level 2 is not sufficient and you may need additional flexibility. We are ready to move on to [Level 3](onboarding_level3.ipynb).